In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile
import scipy.io as sio
import cv2

In [2]:
datapath = '/Users/danielysprague/foco_lab/data'

### Histogram matching

In [3]:
ref_histogram = sio.loadmat('data/ref_histogram.mat')['avg_hist']

In [4]:
def hist_match(sample_image, ref_hist):
    im_flat = sample_image.reshape(-1, sample_image.shape[-1])

    newim = np.zeros(sample_image.shape, 'uint32')

    Amax = np.max(sample_image)

    M = np.zeros((3, int(Amax+1)), 'uint32')

    for l in range(3): # loop through channels
        chan_flat = im_flat[:,l]
        chan_ref = ref_hist[:,l]

        usemax = np.max(chan_flat)

        useedges = np.linspace(0, int(usemax+1), int(usemax+2))

        histcounts, edges = np.histogram(chan_flat, useedges)

        cdf = np.cumsum(histcounts) / chan_flat.size

        sumref = np.cumsum(chan_ref)
        cdf_ref = sumref / np.max(sumref)

        for idx in range(int(usemax+1)):
            ind = np.argmin(np.abs(cdf[idx]-cdf_ref))
            M[l, idx] = ind

        for i in range(sample_image.shape[0]):
            for j in range(sample_image.shape[1]):
                for k in range(sample_image.shape[2]):
                    newim[i,j,k,l] = M[l, sample_image[i,j,k,l]]       
    
    return newim

### Median filtering

In [5]:
def median_filter(im, size):
    if size:
        size = size
    else:
        size=3

    for i in range(im.shape[2]):
        for j in range(im.shape[3]):
            im[:,:,i,j] = cv2.medianBlur(im[:,:,i,j], size)

    return im


### Save images

In [6]:
for folder in ['20240626-12-35-05', '20240626-13-55-40', '20240629-14-14-08', '20240629-14-48-32', '20240629-17-55-55', '20240629-20-22-10']:

    print(folder)

    imfile = sio.loadmat(datapath +'/Manual_annotate/'+folder+'/Composite.mat')

    data = imfile['data']
    channels = [0,1,2,3]
    RGBW = np.squeeze(data[:,:,:,channels])

    newim = median_filter(RGBW, 3)
    newim = hist_match(newim, ref_histogram)

    sio.savemat(datapath+ '/Manual_annotate/'+folder+'/hist_med_image.mat', {'Hist_RGBW':newim})

    imfile = sio.loadmat(datapath+ '/Manual_annotate/'+folder+'/hist_med_image.mat')
    im = np.transpose(imfile['Hist_RGBW'],(2,3,0,1))
    im = im.astype('uint16')
    print(im.dtype)

    tifffile.imwrite(datapath + '/Manual_annotate/'+folder+'/neuroPAL_image.tif', im, imagej = True)


20240626-12-35-05
uint16
20240626-13-55-40
uint16
20240629-14-14-08
uint16
20240629-14-48-32
uint16
20240629-17-55-55
uint16
20240629-20-22-10
uint16


In [20]:
sio.loadmat('../data/Manual_annotate/20240629-20-59-45/neuroPAL_image_ID.mat').keys()

dict_keys(['__header__', '__version__', '__globals__', 'mp_params', 'None', 'version', '__function_workspace__'])

## Update blobs.csv with new annotations

In [ ]:
for folder in os.listdir(datapath+'/Manual_annotate'):
    if folder =='.DS_Store':
        continue
    for f in os.listdir(datapath+'/Manual_annotate/'+folder):
        if f == 'neuroPAL_image.csv':
            npal = pd.read_csv(datapath+'/Manual_annotate/'+folder+'/'+f, skiprows = 7)

        elif f == 'blobs.csv':
            blob = pd.read_csv(datapath+'/Manual_annotate/'+folder+'/'+f, index_col =0)
        elif f == 'neuroPAL_image.mat':
            imfile = sio.loadmat(datapath+'/Manual_annotate/'+folder+'/'+f)
            sz = imfile['data'].shape
            scale = np.asarray(imfile['info']['scale'][0][0]).flatten()

    npal= npal[['Real X (um)', 'Real Y (um)', 'Real Z (um)', 'User ID']]
    npal['Real X (um)'] = round(npal['Real X (um)']/scale[0])
    npal['Real Y (um)'] = round(npal['Real Y (um)']/scale[1])
    npal['Real Z (um)'] = round(npal['Real Z (um)']/scale[2])

    npal['Real X (um)'] = npal['Real X (um)'].astype('int32')
    npal['Real Y (um)'] = npal['Real Y (um)'].astype('int32')
    npal['Real Z (um)'] = npal['Real Z (um)'].astype('int32')

    revx = sz[1]+1- npal['Real X (um)']
    revy = sz[0]+1- npal['Real Y (um)']
    revz = sz[2]+1- npal['Real Z (um)']

    if np.abs(np.mean(np.asarray(npal['Real X (um)']))-np.mean(np.asarray(blob['X'])))>np.abs(np.mean(np.asarray(revx))-np.mean(np.asarray(blob['X']))):
        npal['Real X (um)'] = revx

    if np.abs(np.mean(np.asarray(npal['Real Y (um)']))-np.mean(np.asarray(blob['Y'])))>np.abs(np.mean(np.asarray(revy))-np.mean(np.asarray(blob['Y']))):
        npal['Real Y (um)'] = revy

    if np.abs(np.mean(np.asarray(npal['Real Z (um)']))-np.mean(np.asarray(blob['Z'])))>np.abs(np.mean(np.asarray(revz))-np.mean(np.asarray(blob['Z']))):
        npal['Real Z (um)'] = revz

    npal = npal.rename(columns={'Real X (um)':'X', 'Real Y (um)':'Y', 'Real Z (um)': 'Z'})
        
    blobs_new = pd.merge(blob, npal, how='right', on=['X', 'Y', 'Z'])

    blobs_new['status'] = blobs_new['status'].fillna(1.0)
    blobs_new['prov'] = blobs_new['prov'].fillna('curated')
    blobs_new = blobs_new.drop(columns=['ID'])
    blobs_new = blobs_new.rename(columns={'User ID':'ID'})
    blobs_new['blob'] = np.arange(len(blobs_new))

    blobs_new.to_csv(datapath+'/Manual_annotate/'+folder+'/blobs.csv')
    blobs_new.to_csv(datapath+'/to_upload/'+folder+'/blobs.csv')